In [1]:
import pandas as pd
import unicodedata
import requests

# #send to rest API
import json 
import requests
from requests.auth import HTTPBasicAuth

In [2]:
under_rbi_rate=pd.read_csv('under_RBI_rate.csv')
over_rbi_rate=pd.read_csv('over_RBI_rate.csv')
def_rank = pd.read_csv('../scrapers/def_scraper/batter_rbi_def_rank.csv')
batter_team_2025 = pd.read_csv('../batter_record_1_hit/stats/batters_team_2025.csv')

In [3]:
def_rank.rename(columns={'Rank': 'def_rank'}, inplace=True)

In [4]:
def_rank.rename(columns={'Team': 'team'}, inplace=True)

In [5]:
team_mapping_5 = {
    'CHC': 'CHC',
    'LAD': 'LAD',
    'ARI': 'ARI',
    'PIT': 'PIT',
    'MIA': 'MIA',
    'TOR': 'TOR',
    'BOS': 'BOS',
    'ATH': 'ATH',
    'BAL': 'BAL',
    'SEA': 'SEA',
    'SDP': 'SDP',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'TEX': 'TEX',
    'MIL': 'MIL',
    'WSN': 'WSH',
    'NYY': 'NYY',
    'STL': 'STL',
    'TBR': 'TBR',
    'DET': 'DET',
    'KCR': 'KCR',
    'CLE': 'CLE',
    'COL': 'COL',
    'NYM': 'NYM',
    'CIN': 'CIN',
    'MIN': 'MIN',
    'SFG': 'SFG',
    'HOU': 'HOU',
    'CHW': 'CHW',
    'LAAs': 'LAA'
}

batter_team_2025['team'] = batter_team_2025['team'].map(team_mapping_5)

In [6]:
team_mapping = {
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Chicago Cubs': 'CHC',
    'Colorado Rockies': 'COL',
    'Washington Nationals': 'WSH',
    'Toronto Blue Jays': 'TOR',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'New York Yankees': 'NYY',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'Kansas City Royals': 'KCR',
    'Arizona Diamondbacks': 'ARI',
    'Boston Red Sox': 'BOS',
    'Detroit Tigers': 'DET',
    'Cleveland Guardians': 'CLE',
    'Tampa Bay Rays': 'TBR',
    'San Francisco Giants': 'SFG',
    'Los Angeles Dodgers': 'LAD',
    'Seattle Mariners': 'SEA',
    'Cincinnati Reds': 'CIN',
    'Texas Rangers': 'TEX',
    'Chicago White Sox': 'CHW',
    'Los Angeles Angels': 'LAA',
    'Miami Marlins': 'MIA',
    'New York Mets': 'NYM',
    'Athletics': 'OAK',
    'St. Louis Cardinals': 'STL',
    'San Diego Padres': 'SDP',
    'Houston Astros': 'HOU'
}

def_rank['team'] = def_rank['team'].map(team_mapping)

In [7]:
#fixing the accentes on the names
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [8]:
batter_team_2025['player'] = batter_team_2025['player'].apply(normalize_name)

In [9]:
#cleaning the # and *
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')

batter_team_2025['player'] = batter_team_2025['player'].apply(clean_name_symbols)

In [ ]:
multi_team_flags = ['2TM', '3TM', '4TM', '5TM']
filtered_df_2 = batter_team_2025[~batter_team_2025['team'].isin(multi_team_flags)].copy()


# Step 2: Keep only the LAST team listed for each player (i.e., current team)
batter_team_2025 = filtered_df_2.drop_duplicates(subset='player', keep='last').reset_index(drop=True)

In [11]:
# Set API Key
headers = {
    'X-Api-Key': 'fac7e1a69f2baa47da395bde36dc869c'  # Ensure this is correct
}

# API URL
url = 'https://api.sportsgameodds.com/v2/events/'

next_cursor = None
event_data = []

# Fetch only Over/Under odds for rbi across all players
while True:
    try:
        response = requests.get(
            url,
            headers=headers,
            params={
                "sportID":"BASEBALL",
                "leagueID": "MLB",
                "oddIDs": "batting_RBI-PLAYER_ID-game-ou-over",
                "includeOpposingOddIDs": "true", 
                "oddsAvailable":"true",
                "cursor": next_cursor,
                "include":"teams",
                'started':"false",

            },
        )

        response.raise_for_status()  # Raise an error if the request fails

        data = response.json()
        event_data.extend(data.get("data", []))
        next_cursor = data.get("nextCursor")

        if not next_cursor:
            break

    except requests.exceptions.RequestException as e:
        print("Error fetching events:", e)
        break



In [12]:
event_data

[{'eventID': 'n0C2NflrySgFACc5WKoP',
  'sportID': 'BASEBALL',
  'leagueID': 'MLB',
  'type': 'match',
  'odds': {'batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over': {'oddID': 'batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over',
    'opposingOddID': 'batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under',
    'marketName': 'Christian Yelich Runs Batted In Over/Under',
    'statID': 'batting_RBI',
    'statEntityID': 'CHRISTIAN_YELICH_1_MLB',
    'periodID': 'game',
    'betTypeID': 'ou',
    'sideID': 'over',
    'playerID': 'CHRISTIAN_YELICH_1_MLB',
    'started': False,
    'ended': False,
    'cancelled': False,
    'bookOddsAvailable': True,
    'fairOddsAvailable': True,
    'fairOdds': '+135',
    'bookOdds': '+120',
    'fairOverUnder': '0.5',
    'bookOverUnder': '0.5',
    'openFairOdds': '+135',
    'openBookOdds': '+120',
    'openFairOverUnder': '0.5',
    'openBookOverUnder': '0.5',
    'scoringSupported': True,
    'byBookmaker': {'fanduel': {'odds': '+140',
      'overUnder': '0.

In [13]:
# Initialize an empty list for structured data
odds_list = []

for event in event_data:
    event_id = event.get("eventID", "Unknown Event")
    teams = sorted(set(player_data.get("teamID") for player_data in event.get("players", {}).values() if player_data.get("teamID")))
    
    odds = event.get("odds", {})
    
    for odd_id, odd_object in odds.items():
        base_row = {
            "eventID": event_id,
            "teams": ", ".join(teams),
            "oddID": odd_object.get("oddID"),
            "opposingOddID": odd_object.get("opposingOddID"),
            "marketName": odd_object.get("marketName"),
            "playerID": odd_object.get("statEntityID"),
            "side": odd_object.get("sideID"),
            "line": odd_object.get("bookOverUnder", odd_object.get("fairOverUnder", "N/A")),
            "score": odd_object.get("score", "N/A"),
        }
        
        # Extract bookmaker-specific odds into separate rows
        by_bookmaker = odd_object.get("byBookmaker", {})
        
        for bookmaker, book_data in by_bookmaker.items():
            row = base_row.copy()
            row["bookmaker"] = bookmaker
            row["odds"] = book_data.get("odds", "N/A")
            row["overUnder"] = book_data.get("overUnder", "N/A")
            odds_list.append(row)

# Convert to Pandas DataFrame
df_odds = pd.DataFrame(odds_list)

# Print the DataFrame
df_odds.head(5)

,eventID,teams,oddID,opposingOddID,marketName,playerID,side,line,score,bookmaker,odds,overUnder
0,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under,Christian Yelich Runs Batted In Over/Under,CHRISTIAN_YELICH_1_MLB,over,0.5,N/A,fanduel,+140,0.5
1,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under,Christian Yelich Runs Batted In Over/Under,CHRISTIAN_YELICH_1_MLB,over,0.5,N/A,fanatics,+130,0.5
2,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under,Christian Yelich Runs Batted In Over/Under,CHRISTIAN_YELICH_1_MLB,over,0.5,N/A,espnbet,+115,0.5
3,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under,Christian Yelich Runs Batted In Over/Under,CHRISTIAN_YELICH_1_MLB,over,0.5,N/A,hardrockbet,+120,0.5
4,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under,Christian Yelich Runs Batted In Over/Under,CHRISTIAN_YELICH_1_MLB,over,0.5,N/A,draftkings,+120,0.5


In [14]:
df_odds['bookmaker'].unique()

array(['fanduel', 'fanatics', 'espnbet', 'hardrockbet', 'draftkings',
       'fliff', 'ballybet', 'betrivers', 'betmgm'], dtype=object)

In [15]:
#getting the last 3 letters of the teams collumn to get MLB
df_odds['sport'] = df_odds['teams'].str[-3:]

In [16]:
# Split into two new columns
df_odds[['team_1', 'team_2']] = df_odds['teams'].str.split(', ', expand=True)
# Remove the "_MLB" suffix from both columns using .loc
df_odds.loc[:, 'team_1'] = df_odds['team_1'].str.replace('_MLB', '', regex=False)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].str.replace('_MLB', '', regex=False)
# Drop the original 'teams' column
df_odds = df_odds.drop(columns=['teams'])

In [17]:
df_odds

,eventID,oddID,opposingOddID,marketName,playerID,side,line,score,bookmaker,odds,overUnder,sport,team_1,team_2
0,n0C2NflrySgFACc5WKoP,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under,Christian Yelich Runs Batted In Over/Under,CHRISTIAN_YELICH_1_MLB,over,0.5,N/A,fanduel,+140,0.5,MLB,CLEVELAND_GUARDIANS,MILWAUKEE_BREWERS
1,n0C2NflrySgFACc5WKoP,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under,Christian Yelich Runs Batted In Over/Under,CHRISTIAN_YELICH_1_MLB,over,0.5,N/A,fanatics,+130,0.5,MLB,CLEVELAND_GUARDIANS,MILWAUKEE_BREWERS
2,n0C2NflrySgFACc5WKoP,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under,Christian Yelich Runs Batted In Over/Under,CHRISTIAN_YELICH_1_MLB,over,0.5,N/A,espnbet,+115,0.5,MLB,CLEVELAND_GUARDIANS,MILWAUKEE_BREWERS
3,n0C2NflrySgFACc5WKoP,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under,Christian Yelich Runs Batted In Over/Under,CHRISTIAN_YELICH_1_MLB,over,0.5,N/A,hardrockbet,+120,0.5,MLB,CLEVELAND_GUARDIANS,MILWAUKEE_BREWERS
4,n0C2NflrySgFACc5WKoP,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-over,batting_RBI-CHRISTIAN_YELICH_1_MLB-game-ou-under,Christian Yelich Runs Batted In Over/Under,CHRISTIAN_YELICH_1_MLB,over,0.5,N/A,draftkings,+120,0.5,MLB,CLEVELAND_GUARDIANS,MILWAUKEE_BREWERS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,yfO7V46ops0LswTPjvfe,batting_RBI-CHRISTIAN_KOSS_1_MLB-game-ou-under,batting_RBI-CHRISTIAN_KOSS_1_MLB-game-ou-over,Christian Koss Runs Batted In Over/Under,CHRISTIAN_KOSS_1_MLB,under,0.5,N/A,draftkings,-340,0.5,MLB,ARIZONA_DIAMONDBACKS,SAN_FRANCISCO_GIANTS
2556,yfO7V46ops0LswTPjvfe,batting_RBI-CHRISTIAN_KOSS_1_MLB-game-ou-under,batting_RBI-CHRISTIAN_KOSS_1_MLB-game-ou-over,Christian Koss Runs Batted In Over/Under,CHRISTIAN_KOSS_1_MLB,under,0.5,N/A,espnbet,-375,0.5,MLB,ARIZONA_DIAMONDBACKS,SAN_FRANCISCO_GIANTS
2557,yfO7V46ops0LswTPjvfe,batting_RBI-CHRISTIAN_KOSS_1_MLB-game-ou-under,batting_RBI-CHRISTIAN_KOSS_1_MLB-game-ou-over,Christian Koss Runs Batted In Over/Under,CHRISTIAN_KOSS_1_MLB,under,0.5,N/A,fliff,-380,0.5,MLB,ARIZONA_DIAMONDBACKS,SAN_FRANCISCO_GIANTS
2558,yfO7V46ops0LswTPjvfe,batting_RBI-CHRISTIAN_KOSS_1_MLB-game-ou-under,batting_RBI-CHRISTIAN_KOSS_1_MLB-game-ou-over,Christian Koss Runs Batted In Over/Under,CHRISTIAN_KOSS_1_MLB,under,0.5,N/A,hardrockbet,-350,0.5,MLB,ARIZONA_DIAMONDBACKS,SAN_FRANCISCO_GIANTS


In [18]:
#extract the players name from marketName
df_odds['player'] = df_odds['marketName'].str.extract(r'^(.*?)\s*Runs')

In [19]:
# Define the mapping dictionary
team_mapping = {
    'ARIZONA_DIAMONDBACKS': 'ARI',
    'ATLANTA_BRAVES': 'ATL',
    'BALTIMORE_ORIOLES': 'BAL',
    'BOSTON_RED_SOX': 'BOS',
    'CHICAGO_CUBS': 'CHC',
    'CHICAGO_WHITE_SOX': 'CHW',
    'CINCINNATI_REDS': 'CIN',
    'CLEVELAND_GUARDIANS': 'CLE',
    'COLORADO_ROCKIES': 'COL',
    'DETROIT_TIGERS': 'DET',
    'HOUSTON_ASTROS': 'HOU',
    'KANSAS_CITY_ROYALS': 'KCR',
    'LOS_ANGELES_ANGELS': 'LAA',
    'LOS_ANGELES_DODGERS': 'LAD',
    'MIAMI_MARLINS': 'MIA',
    'MILWAUKEE_BREWERS': 'MIL',
    'MINNESOTA_TWINS': 'MIN',
    'NEW_YORK_METS': 'NYM',
    'NEW_YORK_YANKEES': 'NYY',
    'OAKLAND_ATHLETICS': 'OAK',
    'PHILADELPHIA_PHILLIES': 'PHI',
    'PITTSBURGH_PIRATES': 'PIT',
    'SAN_DIEGO_PADRES': 'SDP',
    'SAN_FRANCISCO_GIANTS': 'SFG',
    'SEATTLE_MARINERS': 'SEA',
    'STLOUIS_CARDINALS': 'STL',
    'TAMPA_BAY_RAYS': 'TBR',
    'TEXAS_RANGERS': 'TEX',
    'TORONTO_BLUE_JAYS': 'TOR',
    'WASHINGTON_NATIONALS': 'WSH'}


# Apply the mapping
df_odds.loc[:, 'team_1'] = df_odds['team_1'].map(team_mapping)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].map(team_mapping)

In [20]:
#making the line collumn an integer
df_odds['overUnder'] = pd.to_numeric(df_odds['overUnder'], errors='coerce')

In [21]:
df_odds['bet']='RBI'

In [22]:
df_odds=df_odds[['player','side','overUnder','bookmaker','odds','sport','team_1','team_2','bet']]

In [23]:
# Merge on the 'player' column
df_odds = df_odds.merge(
    batter_team_2025[['player', 'team']],
    on='player',
    how='left'
)


In [24]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team
0,Christian Yelich,over,0.5,fanduel,+140,MLB,CLE,MIL,RBI,MIL
1,Christian Yelich,over,0.5,fanatics,+130,MLB,CLE,MIL,RBI,MIL
2,Christian Yelich,over,0.5,espnbet,+115,MLB,CLE,MIL,RBI,MIL
3,Christian Yelich,over,0.5,hardrockbet,+120,MLB,CLE,MIL,RBI,MIL
4,Christian Yelich,over,0.5,draftkings,+120,MLB,CLE,MIL,RBI,MIL
...,...,...,...,...,...,...,...,...,...,...
2545,Alek Thomas,under,0.5,draftkings,-360,MLB,ARI,SFG,RBI,ARI
2546,Alek Thomas,under,0.5,espnbet,-400,MLB,ARI,SFG,RBI,ARI
2547,Alek Thomas,under,0.5,fliff,-350,MLB,ARI,SFG,RBI,ARI
2548,Alek Thomas,under,0.5,hardrockbet,-375,MLB,ARI,SFG,RBI,ARI


In [25]:
# Step 1: Identify the "odd one out" team to find the defensive team
df_odds['def_team'] = df_odds.apply(lambda row: row['team_2'] if row['team'] == row['team_1'] else row['team_1'], axis=1)

In [26]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team,def_team
0,Christian Yelich,over,0.5,fanduel,+140,MLB,CLE,MIL,RBI,MIL,CLE
1,Christian Yelich,over,0.5,fanatics,+130,MLB,CLE,MIL,RBI,MIL,CLE
2,Christian Yelich,over,0.5,espnbet,+115,MLB,CLE,MIL,RBI,MIL,CLE
3,Christian Yelich,over,0.5,hardrockbet,+120,MLB,CLE,MIL,RBI,MIL,CLE
4,Christian Yelich,over,0.5,draftkings,+120,MLB,CLE,MIL,RBI,MIL,CLE
...,...,...,...,...,...,...,...,...,...,...,...
2545,Alek Thomas,under,0.5,draftkings,-360,MLB,ARI,SFG,RBI,ARI,SFG
2546,Alek Thomas,under,0.5,espnbet,-400,MLB,ARI,SFG,RBI,ARI,SFG
2547,Alek Thomas,under,0.5,fliff,-350,MLB,ARI,SFG,RBI,ARI,SFG
2548,Alek Thomas,under,0.5,hardrockbet,-375,MLB,ARI,SFG,RBI,ARI,SFG


In [27]:
#drop redundant collumns that are useless
df_odds = df_odds.drop(columns=['team_1', 'team_2'], errors='ignore')

In [28]:
# Rename the team column before merging (if needed)
def_rank = def_rank.rename(columns={"team": "def_team"})

In [29]:
# Merge on the 'player' column
df_odds = df_odds.merge(def_rank, on='def_team', how='left')
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,bet,team,def_team,Unnamed: 0,def_rank
0,Christian Yelich,over,0.5,fanduel,+140,MLB,RBI,MIL,CLE,22,22
1,Christian Yelich,over,0.5,fanatics,+130,MLB,RBI,MIL,CLE,22,22
2,Christian Yelich,over,0.5,espnbet,+115,MLB,RBI,MIL,CLE,22,22
3,Christian Yelich,over,0.5,hardrockbet,+120,MLB,RBI,MIL,CLE,22,22
4,Christian Yelich,over,0.5,draftkings,+120,MLB,RBI,MIL,CLE,22,22
...,...,...,...,...,...,...,...,...,...,...,...
2545,Alek Thomas,under,0.5,draftkings,-360,MLB,RBI,ARI,SFG,6,7
2546,Alek Thomas,under,0.5,espnbet,-400,MLB,RBI,ARI,SFG,6,7
2547,Alek Thomas,under,0.5,fliff,-350,MLB,RBI,ARI,SFG,6,7
2548,Alek Thomas,under,0.5,hardrockbet,-375,MLB,RBI,ARI,SFG,6,7


In [30]:
#making game collumn
df_odds['game'] = df_odds['team'] + ' vs ' + df_odds['def_team']

In [31]:
df_odds2=df_odds.dropna()

In [32]:
df_odds3 = df_odds2.drop(columns=['team', 'def_team'])

In [33]:
df_odds3

,player,side,overUnder,bookmaker,odds,sport,bet,Unnamed: 0,def_rank,game
0,Christian Yelich,over,0.5,fanduel,+140,MLB,RBI,22,22,MIL vs CLE
1,Christian Yelich,over,0.5,fanatics,+130,MLB,RBI,22,22,MIL vs CLE
2,Christian Yelich,over,0.5,espnbet,+115,MLB,RBI,22,22,MIL vs CLE
3,Christian Yelich,over,0.5,hardrockbet,+120,MLB,RBI,22,22,MIL vs CLE
4,Christian Yelich,over,0.5,draftkings,+120,MLB,RBI,22,22,MIL vs CLE
...,...,...,...,...,...,...,...,...,...,...
2545,Alek Thomas,under,0.5,draftkings,-360,MLB,RBI,6,7,ARI vs SFG
2546,Alek Thomas,under,0.5,espnbet,-400,MLB,RBI,6,7,ARI vs SFG
2547,Alek Thomas,under,0.5,fliff,-350,MLB,RBI,6,7,ARI vs SFG
2548,Alek Thomas,under,0.5,hardrockbet,-375,MLB,RBI,6,7,ARI vs SFG


In [34]:
overs = df_odds3[(df_odds3['side'].str.lower() == 'over') & (df_odds3['def_rank'] > 15)]
unders = df_odds3[(df_odds3['side'].str.lower() == 'under') & (df_odds3['def_rank'] < 16)]

In [35]:
overs = overs.merge(over_rbi_rate, on='player', how='left')
unders = unders.merge(under_rbi_rate, on='player', how='left')

In [36]:
overs_2=overs.dropna()
unders_2=unders.dropna()

In [37]:
#making odds collumn numeric
unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')

C:\Users\Joshua\AppData\Local\Temp\ipykernel_90552\2110421890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
C:\Users\Joshua\AppData\Local\Temp\ipykernel_90552\2110421890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')


In [38]:
#making decimal odds
unders_2['decimal_odds'] = unders_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)
overs_2['decimal_odds'] = overs_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_90552\875580270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['decimal_odds'] = unders_2['odds'].apply(
C:\Users\Joshua\AppData\Local\Temp\ipykernel_90552\875580270.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['decimal_odds'] = overs_2['odds'].apply(


In [39]:
#making profit
overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100

C:\Users\Joshua\AppData\Local\Temp\ipykernel_90552\3093691362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
C:\Users\Joshua\AppData\Local\Temp\ipykernel_90552\3093691362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100


In [40]:
def calculate_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_RBI_rate_over']
    elif row['overUnder'] == 1.5:
        rate = row['2_RBI_rate_over']
    elif row['overUnder'] == 2.5:
        rate = row['3_RBI_rate_over']
    elif row['overUnder'] == 3.5:
        rate = row['4_RBI_rate_over']
    else:
        return None  # or np.nan, if you want to leave it blank for other lines

    # Expected Value formula
    return (rate * row['profit']) - ((1 - rate) * 100)

# Apply the formula to each row
overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_90552\882109118.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)


In [41]:
#edge for unders
def calculate_under_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_RBI_rate_under']
    elif row['overUnder'] == 1.5:
        rate = row['2_RBI_rate_under']
    elif row['overUnder'] == 2.5:
        rate = row['3_RBI_rate_under']
    elif row['overUnder'] == 3.5:
        rate = row['4_RBI_rate_under']
    else:
        return None  # or np.nan for unsupported lines

    return ((1 - rate) * row['profit']) - (rate * 100)

# Apply to each row
unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_90552\2277347561.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)


In [42]:
overs_2 = overs_2[overs_2['edge'] > 20]

In [43]:
unders_2 = unders_2[unders_2['edge'] > 20]

In [44]:
#merging overs_2 on top of unders_2
final_df = pd.concat([overs_2, unders_2], ignore_index=True)

In [45]:
final_df=final_df[['player','side','overUnder','bookmaker','odds','sport','bet','edge','game']]

In [46]:
final_df['bet'] = final_df['side'].astype(str) + ' ' + final_df['overUnder'].astype(str) + ' ' + final_df['bet'].astype(str)


In [47]:
final_df = final_df.rename(columns={'bookmaker': 'book'})

In [48]:
final_df['edge'] = final_df['edge'] / 2

In [49]:
final_df['edge'] = final_df['edge'].round(1)

In [50]:
final_df=final_df[['book','sport','player','game','bet','odds','edge']]

In [51]:
final_df = final_df.sort_values(by='edge', ascending=False).reset_index(drop=True)


In [52]:
# #changing book names
# team_mapping_3 = {
#     'betonline': 'BetOnline',
#     'bovada': 'Bovada',
#     'fanatics': 'Fanatics',
#     'hardrockbet': 'Hard Rock Bet',
#     'sporttrade': 'Sporttrade',
#     'balleybet': 'Balley Bet',
#     'betmgm': 'BetMGM',
#     'draftkings': 'DraftKings',
#     'ceasers': 'Ceasers',
#     'fanduel': 'FanDuel',
#     'fliff': 'Fliff',
#     'espnbet': 'ESPNBET',
#     'betrivers': 'BetRivers'
# }

# final_df['book'] = final_df['book'].map(team_mapping_3)

In [53]:
#putting + in front of the ones that dont have it
final_df['odds'] = final_df['odds'].apply(lambda x: f"+{x}" if x > 0 else str(x))

final_df

,book,sport,player,game,bet,odds,edge
0,fanduel,MLB,Dane Myers,MIA vs CHC,over 0.5 RBI,+280,28.2
1,betrivers,MLB,CJ Abrams,WSH vs ATL,over 0.5 RBI,+200,25.0
2,ballybet,MLB,CJ Abrams,WSH vs ATL,over 0.5 RBI,+200,25.0
3,ballybet,MLB,Anthony Volpe,NYY vs SEA,over 0.5 RBI,+210,21.0
4,betrivers,MLB,Anthony Volpe,NYY vs SEA,over 0.5 RBI,+210,21.0
...,...,...,...,...,...,...,...
80,hardrockbet,MLB,Jackson Merrill,SDP vs LAA,over 0.5 RBI,+170,10.3
81,betmgm,MLB,Jackson Merrill,SDP vs LAA,over 0.5 RBI,+170,10.3
82,espnbet,MLB,Connor Wong,BOS vs DET,under 0.5 RBI,-275,10.2
83,hardrockbet,MLB,Connor Wong,BOS vs DET,under 0.5 RBI,-275,10.2


In [54]:
# # Convert DataFrame to JSON (Python list of dicts)
# data_json = json.loads(final_df.to_json(orient="records"))

# # Endpoint URL
# url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_total_RBI"

# # Make sure this is your FULL WordPress login email (not just 'josheskragmailcom')
# username = "josheskra@gmail.com"

# # This must be your app password EXACTLY as shown when created — include all spaces
# password = "BFgx z2iv nW9I voXn 1X51 fjRq"

# # Headers
# headers = {
#     "Content-Type": "application/json"
# }

# # POST request
# response = requests.post(
#     url,
#     headers=headers,
#     auth=HTTPBasicAuth(username, password),
#     json=data_json
# )

# # Debug
# print("Status Code:", response.status_code)
# print("Response Text:", response.text)

In [55]:
import json
import requests

# Convert your DataFrame to JSON
data_json = json.loads(final_df.to_json(orient="records"))

# Endpoint URL
url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_RBI"

# Headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request without authentication
response = requests.post(url, headers=headers, json=data_json)

# Debugging output
print("Status Code:", response.status_code)
print("Response Text:", response.text)


Status Code: 200
Response Text: {"message":"Data received","data":[{"book":"fanduel","sport":"MLB","player":"Dane Myers","game":"MIA vs CHC","bet":"over 0.5 RBI","odds":"+280","edge":28.2},{"book":"betrivers","sport":"MLB","player":"CJ Abrams","game":"WSH vs ATL","bet":"over 0.5 RBI","odds":"+200","edge":25},{"book":"ballybet","sport":"MLB","player":"CJ Abrams","game":"WSH vs ATL","bet":"over 0.5 RBI","odds":"+200","edge":25},{"book":"ballybet","sport":"MLB","player":"Anthony Volpe","game":"NYY vs SEA","bet":"over 0.5 RBI","odds":"+210","edge":21},{"book":"betrivers","sport":"MLB","player":"Anthony Volpe","game":"NYY vs SEA","bet":"over 0.5 RBI","odds":"+210","edge":21},{"book":"betmgm","sport":"MLB","player":"Kyle Manzardo","game":"CLE vs MIL","bet":"over 0.5 RBI","odds":"+200","edge":20.6},{"book":"fanduel","sport":"MLB","player":"Austin Wells","game":"NYY vs SEA","bet":"over 0.5 RBI","odds":"+180","edge":20},{"book":"hardrockbet","sport":"MLB","player":"Kyle Manzardo","game":"CLE vs

In [56]:
# # Convert DataFrame to JSON (ensure it's a Python list, not a string)
# data_json = json.loads(final_df.to_json(orient="records"))  # ✅ Converts string to Python object

# # WordPress API URL
# url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_RBI"

# # Authentication
# # auth = HTTPBasicAuth("josheskragmailcom", "v!(d$mQZnAxSqek!1iAD$d!Q")
# auth = HTTPBasicAuth("josheskragmailcom", "UhiM WLwm dEOK T171 RUkT Sw2F")
# # Headers
# headers = {
#     "Content-Type": "application/json"
# }

# # Send POST request (requests will handle JSON encoding automatically)
# response = requests.post(url, headers=headers, auth=auth, json=data_json)  # ✅ FIX: Use json= instead of data=

# # Debugging Output
# print("Status Code:", response.status_code)
# print("Response Text:", response.text)

In [ ]:
#YOU CAN DELETE ALL OF THIS THIS IS FOR WHEN I KEEP TRACK OF THE BETS LOCALLY

In [57]:
# Step 1: Sort to ensure reproducibility in tie cases
df_sorted = final_df.sort_values('edge')

# Step 2: Define a function to get the row with median edge for each group
def get_median_row(group):
    median_edge = group['edge'].median()
    # Get the row closest to the median (or first in case of exact match/ties)
    return group.iloc[(group['edge'] - median_edge).abs().argsort().iloc[0]]

# Step 3: Apply it to each group
df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)

# Step 4: Reset index if needed
df_filtered.reset_index(drop=True, inplace=True)
df_filtered

C:\Users\Joshua\AppData\Local\Temp\ipykernel_90552\1266780210.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)


,book,sport,player,game,bet,odds,edge
0,betmgm,MLB,Anthony Volpe,NYY vs SEA,over 0.5 RBI,+195,17.6
1,hardrockbet,MLB,Austin Wells,NYY vs SEA,over 0.5 RBI,+165,16.2
2,fanatics,MLB,CJ Abrams,WSH vs ATL,over 0.5 RBI,+175,18.8
3,betmgm,MLB,Connor Wong,BOS vs DET,under 0.5 RBI,-275,10.2
4,fanduel,MLB,Dane Myers,MIA vs CHC,over 0.5 RBI,+280,28.2
5,ballybet,MLB,Heliot Ramos,SFG vs ARI,over 0.5 RBI,+190,15.9
6,fanduel,MLB,Jackson Chourio,MIL vs CLE,over 0.5 RBI,+150,12.5
7,betmgm,MLB,Jackson Merrill,SDP vs LAA,over 0.5 RBI,+170,10.3
8,betrivers,MLB,Keibert Ruiz,WSH vs ATL,over 0.5 RBI,+185,11.6
9,ballybet,MLB,Kyle Manzardo,CLE vs MIL,over 0.5 RBI,+188,17.8


In [58]:
# Make sure odds column is numeric
df_filtered['odds'] = pd.to_numeric(df_filtered['odds'], errors='coerce')
# Convert American odds to decimal odds
def american_to_decimal(odds):
    if pd.isna(odds):
        return None
    if odds > 0:
        return round((odds / 100) + 1, 3)
    else:
        return round((100 / abs(odds)) + 1, 3)

# Apply conversion
df_filtered['decimal_odds'] = df_filtered['odds'].apply(american_to_decimal)

# Check it out
print(df_filtered[['odds', 'decimal_odds']].head())

   odds  decimal_odds
0   195         2.950
1   165         2.650
2   175         2.750
3  -275         1.364
4   280         3.800


In [59]:
df_filtered['profit'] = (df_filtered['decimal_odds'] * 100) - 100
df_filtered

,book,sport,player,game,bet,odds,edge,decimal_odds,profit
0,betmgm,MLB,Anthony Volpe,NYY vs SEA,over 0.5 RBI,195,17.6,2.950,195.0
1,hardrockbet,MLB,Austin Wells,NYY vs SEA,over 0.5 RBI,165,16.2,2.650,165.0
2,fanatics,MLB,CJ Abrams,WSH vs ATL,over 0.5 RBI,175,18.8,2.750,175.0
3,betmgm,MLB,Connor Wong,BOS vs DET,under 0.5 RBI,-275,10.2,1.364,36.4
4,fanduel,MLB,Dane Myers,MIA vs CHC,over 0.5 RBI,280,28.2,3.800,280.0
5,ballybet,MLB,Heliot Ramos,SFG vs ARI,over 0.5 RBI,190,15.9,2.900,190.0
6,fanduel,MLB,Jackson Chourio,MIL vs CLE,over 0.5 RBI,150,12.5,2.500,150.0
7,betmgm,MLB,Jackson Merrill,SDP vs LAA,over 0.5 RBI,170,10.3,2.700,170.0
8,betrivers,MLB,Keibert Ruiz,WSH vs ATL,over 0.5 RBI,185,11.6,2.850,185.0
9,ballybet,MLB,Kyle Manzardo,CLE vs MIL,over 0.5 RBI,188,17.8,2.880,188.0
